In [1]:
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras import layers, models, optimizers, losses, metrics
from tensorflow.keras.preprocessing import image
import os
from skimage.feature import hog
import joblib

load_model_good=tf.keras.models.load_model(r"E:\KPDL-CK\models\Tuan_Tu\tuan_tu_model_good.h5")
load_model_metal=tf.keras.models.load_model(r"E:\KPDL-CK\models\Tuan_Tu\tuan_tu_model_metal.h5")
load_model_3 = joblib.load(r"E:\KPDL-CK\models\Tuan_Tu\svm_model_3.pkl")



C:\Users\quoch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
def predict_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("Lỗi: Không thể đọc ảnh.")
        return None
    
    # Tính toán đặc trưng HOG
    feature, _ = hog(
        img,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys',
        visualize=True,
        feature_vector=True
    )
    
    # Chuyển feature về dạng numpy array
    feature = np.array(feature).reshape(1, -1)  # Reshape về (1, số lượng đặc trưng)
    
    # Dự đoán
    predicted_class = load_model_3.predict(feature)[0]
    # print(f"Dự đoán lớp: {predicted_class}")
    return predicted_class

In [ ]:
path = r"E:\KPDL-CK\data\test\500"
list_path=os.listdir(path)
count_good=0
count_metal=0
count_hole=0
count_thread=0
count_cut=0
for i in list_path:
    img_path=os.path.join(path,i)

    # Load ảnh với kích thước IMG_SIZE và chế độ grayscale như khi training
    img = image.load_img(img_path, target_size=(64,64), color_mode="grayscale")
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Thêm batch dimension

    # Dự đoán
    pred = (load_model_good.predict(img_array)>0.5).astype(int)[0][0]
    if pred == 1:
        pred = (load_model_metal.predict(img_array)>0.5).astype(int)[0][0]
        if pred == 1:
            pred=predict_image(img_path)
            if pred=="cut":
                count_cut+=1
            elif pred=="hole":
                count_hole+=1
            else:
                count_thread+=1
        else:
            count_metal+=1
    else:
        count_good+=1


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━

In [ ]:
print("Total Good: ",count_good)
print("Total Metal: ",count_metal) 
print("Total Hole: ",count_hole)
print("Total Thread: ",count_thread) 
print("Total Cut: ",count_cut)

Total Good:  113
Total Metal:  89
Total Hole:  124
Total Thread:  98
Total Cut:  76
